# 03_prediccion_precios.ipynb

## Predicción de precios para nuevas piezas
Este notebook permite predecir automáticamente el precio estimado de nuevas piezas plásticas a partir de sus características técnicas (como peso, tiempo de ciclo, cavidades, material, etc.), utilizando un modelo previamente entrenado con datos históricos.

In [1]:
# Importar las librerías
import pandas as pd
import joblib
from sklearn.ensemble import GradientBoostingRegressor

### 1. Carga de datos históricos
Se carga el dataset histórico para entrenar el modelo final.

In [2]:
# Cargar datos históricos
hist_file = 'piezas_con_precios.csv'
df_hist = pd.read_csv(hist_file, sep=';')
X_hist = df_hist.drop(columns=['referencia', 'descripcion', 'precio_final'])
y_hist = df_hist['precio_final']
X_hist_encoded = pd.get_dummies(X_hist, columns=['material'])

### 2. Entrenamiento del modelo elegido
Se carga el modelo Gradient Boosting (optimizado) con todos los datos históricos disponibles.

In [3]:
modelo_final = joblib.load("modelo_gradient_boosting_optimizado.pkl")
modelo_final.fit(X_hist_encoded, y_hist)

GradientBoostingRegressor(learning_rate=0.2, n_estimators=200, random_state=42)

### 3. Carga de nuevas piezas
Se leen las piezas nuevas cuyo precio deseamos estimar.

In [4]:
# Cargar nuevas piezas
df_nuevas = pd.read_csv('nuevas_piezas.csv', sep=';')
df_nuevas

,referencia,descripcion,material,peso_(kg),tiempo_ciclo_(s),numero_cavidades,tonelaje_maquina
0,REF201,Pieza TEST A,PC,1.2,65,1,1200
1,REF202,Pieza TEST B,PP UV,0.6,42,2,500
2,REF203,Pieza TEST C,ABS,2.8,85,2,1800
3,REF204,Pieza TEST D,PMMA,4.2,95,1,2700
4,REF205,Pieza TEST E,PP-GF30,3.0,80,2,2000
5,REF206,Pieza TEST F,PP,0.9,50,1,500
6,REF207,Pieza TEST G,PE,1.5,60,1,650
7,REF208,Pieza TEST H,POM,2.5,75,1,1000
8,REF209,Pieza TEST I,PMMA,6.2,100,1,2700
9,REF210,Pieza TEST J,ABS,3.5,80,1,1200


In [5]:
X_nuevas = df_nuevas.drop(columns=['referencia', 'descripcion'])
X_nuevas_encoded = pd.get_dummies(X_nuevas)

### 4. Alineación de columnas
Nos aseguramos de que las columnas de las nuevas piezas coincidan con las del conjunto histórico, rellenando con ceros donde sea necesario.

In [6]:
X_nuevas_encoded = X_nuevas_encoded.reindex(columns=X_hist_encoded.columns, fill_value=0)

### 5. Predicción y exportación de precios
Se estiman los precios usando el modelo entrenado y se exportan en un .csv.

In [7]:
# Exportar .csv con resultados
df_nuevas['precio_estimado'] = modelo_final.predict(X_nuevas_encoded).round(4)
df_nuevas.to_csv('nuevas_piezas_con_precios.csv', sep=';', index=False)
print("Se ha generado el archivo 'nuevas_piezas_con_precios.csv' con los precios estimados.")

Se ha generado el archivo 'nuevas_piezas_con_precios.csv' con los precios estimados.


In [8]:
# Mostrar vista previa de los resultados
df_nuevas

,referencia,descripcion,material,peso_(kg),tiempo_ciclo_(s),numero_cavidades,tonelaje_maquina,precio_estimado
0,REF201,Pieza TEST A,PC,1.2,65,1,1200,6.4008
1,REF202,Pieza TEST B,PP UV,0.6,42,2,500,2.2638
2,REF203,Pieza TEST C,ABS,2.8,85,2,1800,10.4536
3,REF204,Pieza TEST D,PMMA,4.2,95,1,2700,20.9567
4,REF205,Pieza TEST E,PP-GF30,3.0,80,2,2000,11.9155
5,REF206,Pieza TEST F,PP,0.9,50,1,500,3.8079
6,REF207,Pieza TEST G,PE,1.5,60,1,650,5.0948
7,REF208,Pieza TEST H,POM,2.5,75,1,1000,12.8148
8,REF209,Pieza TEST I,PMMA,6.2,100,1,2700,29.8759
9,REF210,Pieza TEST J,ABS,3.5,80,1,1200,13.1865
